In [1]:
!pip install -q underthesea
!pip install -q rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 82.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# Create a VQA Model using CNN and LSTM

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import os
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision.models as models
from torchvision.models import resnet50, ResNet50_Weights
from torch.optim import AdamW
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import seaborn as sns
import pandas as pd
from transformers import AutoTokenizer, ViTFeatureExtractor, AutoModel, CLIPImageProcessor, AutoTokenizer, AutoImageProcessor, ViTModel
from transformers import BlipProcessor, BlipForConditionalGeneration
from datasets import load_dataset
from datasets import load_dataset, Features, Value
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import os
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from collections import Counter
from underthesea import word_tokenize, text_normalize
import re
import string
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from nltk.translate.bleu_score import corpus_bleu
from sklearn.model_selection import train_test_split
from rouge_score import rouge_scorer

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 1
MAX_LEN = 100
EPOCHS = 2

2025-05-12 02:29:12.132611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747016952.317432      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747016952.370359      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Data visulization

In [3]:
root_dir = "/kaggle/input/data-nlp-bai-2/data"
data_1 = pd.read_csv(f"{root_dir}/data_1.csv")
data_2 = pd.read_csv(f"{root_dir}/data_2.csv")
data_3 = pd.read_csv(f"{root_dir}/data_3.csv")
data_4 = pd.read_csv(f"{root_dir}/data_4.csv")
data_5 = pd.read_csv(f"{root_dir}/data_5.csv")
data_6 = pd.read_csv(f"{root_dir}/data_6.csv")
data_7 = pd.read_csv(f"{root_dir}/data_7.csv")
data_8 = pd.read_csv(f"{root_dir}/data_8.csv")
data_9 = pd.read_csv(f"{root_dir}/data_9.csv")
df_1 = pd.concat([data_1, data_2, data_3, data_4, data_5, data_6, data_7, data_8, data_9])

In [4]:
root_dir = '/kaggle/input/data-nlp-bai-2/img/img'
df = df_1.copy()
# df = df.iloc[0:10000]
df = df.rename({"image_path": "image_id"}, axis = 1)
df["image_path"] = df['image_id'].apply(lambda x: os.path.join(root_dir, x))
df["image_id"] = df["image_id"].str.split(".").str[0]
df = df.dropna(subset=['question','answer']).copy()
df['question'] = df['question'].astype(str)
df['answer'] = df['answer'].astype(str)
df.drop(['description'], axis = 1, inplace = True)
df.reset_index(inplace=True)
df

,index,image_id,question,answer,image_path
0,0,0,Cuốn sách trong ảnh thuộc bộ môn học nào?,Cuốn sách trong ảnh thuộc bộ môn Ngữ văn.,/kaggle/input/data-nlp-bai-2/img/img/0.jpg
1,1,0,Cuốn sách dành cho học sinh lớp nào?,Cuốn sách dành cho học sinh lớp 12.,/kaggle/input/data-nlp-bai-2/img/img/0.jpg
2,2,0,Cuốn sách được in lần thứ mấy?,Cuốn sách được in lần thứ ba.,/kaggle/input/data-nlp-bai-2/img/img/0.jpg
3,3,0,Ai là người chủ biên phần tiếng Việt của cuốn ...,Bùi Minh Toàn là người chủ biên phần tiếng Việ...,/kaggle/input/data-nlp-bai-2/img/img/0.jpg
4,4,0,Cuốn sách do đơn vị nào xuất bản?,Cuốn sách được xuất bản bởi Nhà xuất bản Giáo ...,/kaggle/input/data-nlp-bai-2/img/img/0.jpg
...,...,...,...,...,...
259089,19091,51855,Câu hỏi 1 đề cập đến vấn đề gì?,Câu hỏi 1 đề cập đến khả năng lưu trữ văn bản ...,/kaggle/input/data-nlp-bai-2/img/img/51855.jpg
259090,19092,51855,Câu hỏi 2 đề cập đến các biện pháp bảo vệ dữ l...,Câu hỏi 2 đề cập đến 5 biện pháp bảo vệ dữ liệ...,/kaggle/input/data-nlp-bai-2/img/img/51855.jpg
259091,19093,51855,Hình ảnh minh họa cho câu hỏi 1 là gì?,Hình ảnh minh họa cho câu hỏi 1 là 4 khung tho...,/kaggle/input/data-nlp-bai-2/img/img/51855.jpg
259092,19094,51855,Hình ảnh minh họa cho phần bài tập vận dụng là...,Hình ảnh minh họa cho phần bài tập vận dụng là...,/kaggle/input/data-nlp-bai-2/img/img/51855.jpg


In [5]:
df = df[df['image_path'] != '/kaggle/input/data-nlp-bai-2/img/img/6803.jpg']
df = df[df['image_path'] != '/kaggle/input/data-nlp-bai-2/img/img/6801.jpg']
df[df["image_id"] == "6803"]

,index,image_id,question,answer,image_path


In [6]:
df[df["image_id"] == "6801"]

,index,image_id,question,answer,image_path


In [7]:
def normalize_text(text):
    clean_text = text.lower()

    replacements = {
        "nxb": "nhà xuất bản",
    }

    for old, new in replacements.items():
        clean_text = clean_text.replace(old, new)

    # Xóa ký tự đặc biệt VD (&nbsp;)
    clean_text = re.sub(r'&\S+;', ' ', clean_text)

    # Xóa emoji
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               "]+", flags=re.UNICODE)
    clean_text = re.sub(emoji_pattern, " ", clean_text)

    # Xóa ký tự đặc biệt không gõ được
    special_chars = ["“", "”", "…", "•", "–", "’", " ️", "✅", "✓", " ̂́ ̃ ̣̂ ", "‘", "●", "‒", "➤",
                     "★", "ღ", "✪", "‎", "➦", "×", "", "✿", "☆", "◤", "◕", "❁", "‿",
                     "❀", "■", "█", "☛", "⑴⒪⑵⑵⑴⑺", "►", "°", "»", "ø", "➽", "", "✧", "✽", "*",
                     "➫", "【", "】", "⇒", "卐", "♛", "±", "∞", "②", "⑥", "①", "⑦", "➋", "➊", "➌",
                     "✓", "™", "®", "", ""]
    for char in special_chars:
        clean_text = clean_text.replace(char, " ")

    # Bỏ các ký tự đặc biệt (dấu câu)
    clean_text = ''.join(' ' if char in string.punctuation else char for char in clean_text)

    # Loại bỏ khoảng trắng thừa
    clean_text = re.sub(r"\s+", " ", clean_text)
    clean_text = re.sub(r"^[\s]", "", clean_text)
    clean_text = re.sub(r"[\s]$", "", clean_text)

    # Đảm bảo dấu ở đúng chữ (ví dụ: oà, uý)
    clean_text = text_normalize(clean_text)

    # Tách câu thành từ
    clean_text = word_tokenize(clean_text, format="text")

    return clean_text

def apply_normalize_text_to_dataframe(df, columns):
    for col in columns:
        df[col] = df[col].apply(lambda x: normalize_text(str(x)))
    return df

In [ ]:
columns_to_process = ['question', 'answer']
df = apply_normalize_text_to_dataframe(df, columns_to_process)
df

# Data visualization

In [ ]:
samples = df.sample(n=random.randint(2, min(2, len(df)))).reset_index(drop=True)

for i, row in samples.iterrows():
    image = row['image_path']
    question = row['question']
    answer = row['answer']

    try:
        # Mở và hiển thị hình ảnh
        img = Image.open(image)

        plt.figure(figsize=(10, 10))
        plt.imshow(img)
        plt.title(f"Câu hỏi: {question}\nCâu trả lời: {answer}", fontsize=10)
        plt.axis('off')  # Ẩn trục để hiển thị gọn gàng hơn
        plt.show()
    except FileNotFoundError:
        print(f"Không tìm thấy hình ảnh: {image}")
    except Exception as e:
        print(f"Lỗi khi tải hình ảnh {image}: {e}")

In [ ]:
def get_sentence_length(text):
    if not isinstance(text, str):
        return 0
    tokens = word_tokenize(text)
    return len(tokens)

In [ ]:
question_lengths = df['question'].apply(get_sentence_length).tolist()
answer_lengths = df['answer'].apply(get_sentence_length).tolist()

# Thiết lập bố cục subplot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# Biểu đồ 1: Độ dài câu hỏi
ax1.hist(question_lengths, bins=20, color='skyblue', edgecolor='black')
ax1.set_title('Phân bố độ dài câu hỏi')
ax1.set_xlabel('Số từ')
ax1.set_ylabel('Số lượng câu')
ax1.grid(True, alpha=0.3)

# Biểu đồ 2: Độ dài câu trả lời
ax2.hist(answer_lengths, bins=20, color='lightgreen', edgecolor='black')
ax2.set_title('Phân bố độ dài câu trả lời')
ax2.set_xlabel('Số từ')
ax2.grid(True, alpha=0.3)

# Tùy chỉnh khoảng cách giữa các subplot
plt.tight_layout()

In [ ]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    return word_tokenize(text, format="text")

In [ ]:
# Tính độ tương đồng cosine
# Chuẩn hóa văn bản cho TF-IDF
questions_preprocessed = df['question'].apply(preprocess_text).tolist()
answers_preprocessed = df['answer'].apply(preprocess_text).tolist()

# Kết hợp câu hỏi và câu trả lời để tạo TF-IDF vector
corpus = questions_preprocessed + answers_preprocessed
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

# Tách TF-IDF matrix thành phần cho câu hỏi và câu trả lời
question_tfidf = tfidf_matrix[:len(questions_preprocessed)]
answer_tfidf = tfidf_matrix[len(questions_preprocessed):]

# Tính cosine similarity cho từng cặp câu hỏi-câu trả lời
similarities = [cosine_similarity(question_tfidf[i], answer_tfidf[i])[0][0] for i in range(len(questions_preprocessed))]

# Vẽ biểu đồ histogram cho độ tương đồng
plt.figure(figsize=(8, 5))
plt.hist(similarities, bins=20, color='lightcoral', edgecolor='black')
plt.title('Phân bố độ tương đồng câu hỏi - câu trả lời')
plt.xlabel('Cosine Similarity')
plt.ylabel('Số lượng cặp')
plt.grid(True, alpha=0.3)

# Tùy chỉnh bố cục
plt.tight_layout()

# Add data gen

In [ ]:
root_dir = "/kaggle/input/data-generative/Data_genarative"
df_list = []

for file_name in os.listdir(root_dir):
    if file_name.endswith('.csv'):
        file_path = os.path.join(root_dir, file_name)
        try:
            df_gen = pd.read_csv(file_path)
            df_list.append(df_gen)
            print(f"Đã đọc file: {file_name}")
        except Exception as e:
            print(f"Lỗi khi đọc file {file_name}: {e}")

df_gen1 = pd.concat(df_list, ignore_index=True)

In [ ]:
root_dir = '/kaggle/input/data-nlp-bai-2/img/img'
df_gen1 = df_gen1.rename({"image_path": "image_id"}, axis = 1)
df_gen1["image_path"] = df_gen1['image_id'].apply(lambda x: os.path.join(root_dir, x))
df_gen1["image_id"] = df_gen1["image_id"].str.split(".").str[0]
df_gen1 = df_gen1.dropna(subset=['question','answer']).copy()
df_gen1['question'] = df_gen1['question'].astype(str)
df_gen1['answer'] = df_gen1['answer'].astype(str)
df_gen1.drop(['description'], axis = 1, inplace = True)
df_gen1.reset_index(inplace=True)
df_gen1

In [ ]:
columns_to_process = ['question', 'answer']
df_gen1 = apply_normalize_text_to_dataframe(df_gen1, columns_to_process)
df_gen1

In [ ]:
df_1 = df.copy()
df_gen_2 = df_gen1.copy()

In [ ]:
df = pd.concat([df_1, df_gen_2], ignore_index=True)

df = df.sort_values(by=['image_path', 'answer'], ignore_index=True)
df.drop_duplicates(inplace=True, ignore_index=True)
df.drop(["index"], axis = 1, inplace = True)
df.reset_index(inplace=True)
new_columns = ["index",'image_id', "image_path", 'question', 'answer']
df = df[new_columns]
df

In [ ]:
question_lengths = df['question'].apply(get_sentence_length).tolist()
answer_lengths = df['answer'].apply(get_sentence_length).tolist()

# Thiết lập bố cục subplot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# Biểu đồ 1: Độ dài câu hỏi
ax1.hist(question_lengths, bins=20, color='skyblue', edgecolor='black')
ax1.set_title('Phân bố độ dài câu hỏi')
ax1.set_xlabel('Số từ')
ax1.set_ylabel('Số lượng câu')
ax1.grid(True, alpha=0.3)

# Biểu đồ 2: Độ dài câu trả lời
ax2.hist(answer_lengths, bins=20, color='lightgreen', edgecolor='black')
ax2.set_title('Phân bố độ dài câu trả lời')
ax2.set_xlabel('Số từ')
ax2.grid(True, alpha=0.3)

# Tùy chỉnh khoảng cách giữa các subplot
plt.tight_layout()

In [ ]:
def preprocess_text(text):
    # Chuyển thành chữ thường
    text = text.lower()
    # Loại bỏ dấu câu
    text = re.sub(f'[{string.punctuation}]', ' ', text)
    # Thay dấu '_' bằng khoảng trắng
    text = text.replace('_', ' ')
    # Tokenize bằng underthesea
    text = word_tokenize(text, format="text")
    return text

In [ ]:
def calculate_similarity(questions):
    if len(questions) < 2:
        return []  # Không tính nếu chỉ có 1 câu hỏi
    
    # Tiền xử lý các câu hỏi
    processed_questions = [preprocess_text(q) for q in questions]
    
    # Vector hóa bằng TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_questions)
    
    # Tính độ tương đồng cosine
    similarity_matrix = cosine_similarity(tfidf_matrix)
    
    # Lưu kết quả cho từng cặp câu hỏi
    similarities = []
    for i in range(len(questions)):
        for j in range(i + 1, len(questions)):
            similarities.append({
                'question_1': questions[i],
                'question_2': questions[j],
                'similarity': similarity_matrix[i][j]
            })
    
    return similarities

In [ ]:
grouped = df.groupby('answer')

# Lưu kết quả độ tương đồng
all_similarities = []

# Duyệt qua từng nhóm có cùng 'answer'
for answer, group in grouped:
    if len(group) > 1:  # Chỉ xét các nhóm có từ 2 câu hỏi trở lên
        questions = group['question'].tolist()
        similarities = calculate_similarity(questions)
        for sim in similarities:
            sim['answer'] = answer  # Gắn câu trả lời vào kết quả
            all_similarities.append(sim)

# Chuyển kết quả thành DataFrame
similarity_df = pd.DataFrame(all_similarities)

# Sắp xếp theo độ tương đồng (giảm dần)
# similarity_df = similarity_df.sort_values(by='similarity', ascending=False)

In [ ]:
similarity_df

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(similarity_df['similarity'], bins=20, kde=True)
plt.title('Phân bố độ tương đồng giữa các cặp câu hỏi')
plt.xlabel('Độ tương đồng (Cosine Similarity)')
plt.ylabel('Số lượng cặp câu hỏi')
plt.grid(True)
plt.savefig('similarity_histogram.png')
plt.show()

In [ ]:
max_group = grouped.size().idxmax()
group = grouped.get_group(max_group)
questions = group['question'].tolist()

if len(questions) > 1:
    processed_questions = [preprocess_text(q) for q in questions]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_questions)
    similarity_matrix = cosine_similarity(tfidf_matrix)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(similarity_matrix, annot=True, cmap='Blues', xticklabels=questions, yticklabels=questions)
    plt.title(f'Ma trận độ tương đồng cho nhóm có answer: "{max_group}"')
    plt.xlabel('Câu hỏi')
    plt.ylabel('Câu hỏi')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('similarity_heatmap.png')
    plt.show()
else:
    print(f"Nhóm có answer '{max_group}' chỉ có 1 câu hỏi, không vẽ heatmap.")

# Tokenize


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
vocab = tokenizer.get_vocab()
len(vocab)

# Train test split

In [ ]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Kiểm tra kích thước
print("Number of rows in train set:", len(df_train))
print("Number of rows in test set:", len(df_test))
print("Train set columns:", df_train.columns)
print("Test set columns:", df_test.columns)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

# Change form to dataset

In [ ]:
class VQA_dataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        anno_id, img_id, image_path, question, answer = self.data.iloc[idx]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return anno_id, img_id, image, question, answer

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),

    #Thêm 1 vài bước xử lý ảnh để đa dạng hóa dữ liệu - Data Augmentation
    transforms.RandomRotation(30),  # Xoay ảnh ngẫu nhiên trong khoảng ±30 độ
    # transforms.RandomHorizontalFlip(),  # Lật ngang ngẫu nhiên
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Điều chỉnh độ sáng, độ tương phản
    # transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

train_dataset = VQA_dataset(df, transform=transform)
test_dataset = VQA_dataset(df_test, transform=transform)

In [ ]:
random_indices = np.random.choice(len(train_dataset), 2)

for idx in random_indices:
    anno_id, img_id, image, question, answer = train_dataset[idx]
    
    image = image.permute(1, 2, 0).numpy()
    
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.title("Question: " + question + "\nAnswer: " + answer)
    plt.axis('off')
    plt.show()

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=VAL_BATCH_SIZE, shuffle=False)
len(train_loader), len(test_loader)

In [ ]:
anno_id, img_id, image, quesiton, answer = next(enumerate(train_loader))[1]
len(answer), len(image), len(quesiton)

# Using Pretrain for VQA

In [ ]:
image_model_name = "google/vit-base-patch16-224-in21k"
class ImageEmbedding(nn.Module):
    def __init__(self):
        super(ImageEmbedding, self).__init__()
        self.process = AutoImageProcessor.from_pretrained(image_model_name)
        self.model = ViTModel.from_pretrained(image_model_name)
        #self.model = nn.Sequential(*list(self.model.children())[:3])
        
        for param in self.model.parameters():
            param.requires_grad = False

    def forward(self, image, image_ids):
        inputs = self.process(image, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs.to(device))
            
        image_embedding = outputs.last_hidden_state
        return image_embedding, image_ids

In [ ]:
text_model_name = "vinai/phobert-base"
class QuesEmbedding(nn.Module):
    def __init__(self, input_size=768, output_size=768):
        super(QuesEmbedding, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(text_model_name)
        self.phobert = AutoModel.from_pretrained(text_model_name)
        self.lstm = nn.LSTM(input_size, output_size, batch_first=True)

    def forward(self, ques):
        tokenized_input = self.tokenizer(ques, 
                                         return_tensors='pt', 
                                         padding='max_length', 
                                         max_length=MAX_LEN, 
                                         truncation=True, 
                                         add_special_tokens=False)
        ques = self.phobert(**tokenized_input.to(device)).last_hidden_state
        _, (h, _) = self.lstm(ques)
        return h.squeeze(0)

## Test Batch

In [ ]:
image_model = ImageEmbedding().to(device)
ques_model = QuesEmbedding().to(device)

for batch in train_loader:
    anno_ids, img_ids, images, questions, answers = batch
    if torch.cuda.is_available():
        images = images.cuda()
        questions = questions
        anno_ids = anno_ids
    
    with torch.no_grad():
        image_embeddings, att_ids = image_model(images, image_ids=anno_ids)
        ques_embeddings = ques_model(questions)
    break    

In [ ]:
print(image_embeddings.size())
print(att_ids.size())
print(ques_embeddings.size())

In [ ]:
image_embeddings = image_embeddings.reshape(TRAIN_BATCH_SIZE, 768, -1).permute(0, 2, 1)
ques_embeddings = ques_embeddings.unsqueeze(1)
print(image_embeddings.size())
print(ques_embeddings.size())

# Stack Attention Networks

In [ ]:
#Linear để giảm kích thước, không cần tile repeat mà đưa vào san để nhân luôn cuối cùng trả về vector attention mà ko cần phải flatten
class StackAttention(nn.Module):
    def __init__(self, d=768, k=512, dropout=True):
        super(StackAttention, self).__init__()
        self.ff_image = nn.Linear(d, k)
        self.ff_ques = nn.Linear(d, k)
        if dropout:
            self.dropout = nn.Dropout(p=0.5)
        self.ff_attention = nn.Linear(k, 1)

    def forward(self, vi, vq):
        # N * 49 * 768 -> N * 49 * 512
        hi = self.ff_image(vi)
        #print("hi size: ", hi.size())
        # N * 768 -> N * 512 -> N * 1 * 512
        hq = self.ff_ques(vq)
        #print("hq size: ", hq.size())
        # N * 49 * 512
        ha = F.tanh(hi + hq)
        #print("ha1 size: ", ha.size())
        if getattr(self, 'dropout'):
            ha = self.dropout(ha)
        #DONE
        
        # N * 49 * 512 -> N * 49 * 1 -> N * 49
        ha = self.ff_attention(ha).squeeze(dim=2)
        #print("ha2 size: ", ha.size())
        pi = F.softmax(ha)
        # (N * 49 * 1, N * 49 * 768) -> N * 768
        vi_attended = (pi.unsqueeze(dim=2) * vi).sum(dim=1)
        #print("vi~ size: ", vi_attended.size())
        #print("vq size: ", vq.size())
        u = vi_attended + vq.squeeze(1)
        return u

In [ ]:
san_model = StackAttention(d=768, k=512, dropout=True).to(device)
img_text_att = san_model(image_embeddings.to(device), ques_embeddings.to(device))
img_text_att.size()

# Generate Answer

## Answers Embedding

In [ ]:
class AnsEmbedding(nn.Module):
    def __init__(self, input_size=768):
        super(AnsEmbedding, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(text_model_name)
        self.phobert_embed = AutoModel.from_pretrained(text_model_name).embeddings.to(device)

    def forward(self, ans):  
        tokenized_input = self.tokenizer(ans, 
                                         return_tensors='pt',
                                         padding='max_length', 
                                         max_length=MAX_LEN, 
                                         truncation=True, return_attention_mask=False)
        ans = self.phobert_embed(**tokenized_input.to(device))
        return tokenized_input['input_ids'], ans

In [ ]:
ans_model = AnsEmbedding().to(device)

for batch in train_loader:
    anno_id, img_id, images, questions, answers = batch
    if torch.cuda.is_available():
        answers = answers
    with torch.no_grad():
        ans_tokens, answers_embedding = ans_model(answers)
    break   
ans_tokens.size(), answers_embedding.size() 

In [ ]:
print(answers_embedding.size())
print(img_text_att.size())

## Decoder

In [ ]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1] 
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k) 
    if mask is not None:
        scaled += mask 
    attention = F.softmax(scaled, dim=-1) 
    values = torch.matmul(attention, v)
    return values, attention


class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x) 
        x = self.linear2(x)
        return x


class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape)) # 512
        self.beta =  nn.Parameter(torch.zeros(parameters_shape)) # 512

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))] # [-1]
        mean = inputs.mean(dim=dims, keepdim=True) 
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt() 
        y = (inputs - mean) / std 
        out = self.gamma * y  + self.beta  
        return out

class MaskMultiHeadAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model , 3 * d_model) 
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, mask):
        batch_size, sequence_length, d_model = x.size() 
        qkv = self.qkv_layer(x) 
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3) 
        q, k, v = qkv.chunk(3, dim=-1) 
        values, attention = scaled_dot_product(q, k, v, mask) 
        values = values.reshape(batch_size, sequence_length, self.num_heads * self.head_dim) 
        out = self.linear_layer(values) # 30 x 200 x 512
        return out # 30 x 200 x 512


class MultiHeadCrossAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model , 2 * d_model) # 1024
        self.q_layer = nn.Linear(d_model , d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, y, mask=None):
        batch_size, sequence_length, d_model = x.size()
        kv = self.kv_layer(x) 
        q = self.q_layer(y) 
        kv = kv.reshape(batch_size, sequence_length, self.num_heads, 2 * self.head_dim)
        q = q.reshape(batch_size, sequence_length, self.num_heads, self.head_dim) 
        kv = kv.permute(0, 2, 1, 3) 
        q = q.permute(0, 2, 1, 3) 
        k, v = kv.chunk(2, dim=-1) 
        values, attention = scaled_dot_product(q, k, v, mask) 
        values = values.reshape(batch_size, sequence_length, d_model) 
        out = self.linear_layer(values)
        return out  


class DecoderLayer(nn.Module):

    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MaskMultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self, x, y, decoder_mask):
        _y = y
        y = self.self_attention(y, mask=decoder_mask)
        y = self.dropout1(y)
        y = self.norm1(y + _y)

        _y = y # 30 x 200 x 512
        y = self.encoder_decoder_attention(x, y, mask=None)
        y = self.dropout2(y)
        y = self.norm2(y + _y)

        _y = y 
        y = self.ffn(y) 
        y = self.dropout3(y)
        y = self.norm3(y + _y) 
        return y 

class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x, y, mask = inputs
        for module in self._modules.values():
            y = module(x, y, mask)
        return y

class Decoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers=1):
        super().__init__()
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) 
                                          for _ in range(num_layers)])

    def forward(self, x, y, mask):
        y = self.layers(x, y, mask)
        return y

In [ ]:
d_model = 768
num_heads = 8
drop_prob = 0.1
batch_size = 30
max_sequence_length = MAX_LEN
ffn_hidden = 2048
num_layers = 5

x = img_text_att.unsqueeze(1).expand(-1, max_sequence_length, -1).to(device) # 16 * 768 -> 16 * 48 * 1024
y = answers_embedding.to(device) #16 * 48 * 768
mask = torch.full([max_sequence_length, max_sequence_length] , float('-inf'))
mask = torch.triu(mask, diagonal=1).to(device)
decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers).to(device)
out = decoder(x, y, mask).to(device)

In [ ]:
vocab_size = len(vocab)

linear_layer = nn.Linear(d_model, vocab_size).to(device)
output_logits = linear_layer(out)
output_probs = F.softmax(output_logits, dim=2)
output_probs.size()

In [ ]:
from transformers import PhobertTokenizer
tokenizer = PhobertTokenizer.from_pretrained("vinai/phobert-base")

decoder_output = output_probs
predicted_tokens = decoder_output.argmax(dim=2)

def tokens_to_text(tokens, tokenizeFcr):
    return [tokenizer.decode(token.item()) for token in tokens]

for i in range(16):
    generated_text = tokens_to_text(predicted_tokens[i], tokenizer)
    print(f"Generated text for example {i}: {' '.join(generated_text)}")

# VQA Model

In [ ]:
class VQAModel(nn.Module):

    def __init__(self, vocab_size=64001, output_size=768, d_model=768, num_head=8, ffn_hidden=2048, drop_prob=0.3, num_layers=6, num_att_layers=4, mode='train'):
        super(VQAModel, self).__init__()
        self.mode = mode
        self.image_model = ImageEmbedding().to(device)
        self.ques_model = QuesEmbedding().to(device)
        self.ans_model = AnsEmbedding().to(device)
        
        self.san_model = nn.ModuleList(
            [StackAttention(d=d_model, k=512, dropout=True)] * num_att_layers).to(device)
        
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(0.5)
        
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers).to(device)
        
        self.mlp = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(d_model, vocab_size))

    def forward(self, images, questions, answers, anno_ids, mask, mode, max_len=MAX_LEN):
        image_embeddings, att_ids = self.image_model(images.to(device), image_ids=anno_ids)
        if mode == 'train':
            image_embedds = image_embeddings.reshape(TRAIN_BATCH_SIZE, 768, -1).permute(0, 2, 1)
        else:
            image_embedds = image_embeddings.reshape(VAL_BATCH_SIZE, 768, -1).permute(0, 2, 1)
        
        ques_embeddings = self.ques_model(questions)
        ques_embedds = ques_embeddings.unsqueeze(1)
        
        for att_layer in self.san_model:
            att_embedds = att_layer(image_embedds.to(device), ques_embedds.to(device))
        
        att_embedds = self.tanh(att_embedds)
        att_embedds = self.dropout(att_embedds)
        
        #START DECODER
        ans_vocab, ans_embedds = self.ans_model(answers)
        
        x = att_embedds.to(device).unsqueeze(1).expand(-1, max_len, -1).to(device) # 16 * 768 -> 16 * 48 * 768
        y = ans_embedds # 16 * 48 * 768
        #print(x)
        #print(y.size())
        if mask == False:
            out = self.decoder(x, y, mask=None).to(device)
        else:
            mask = torch.full([max_len, max_len] , float('-inf'))
            mask = torch.triu(mask, diagonal=1).to(device)
        
            out = self.decoder(x, y, mask).to(device)
        #END DECODER
        #print(out)
        output_logits = self.mlp(out)
        return output_logits, ans_vocab

# Train

In [ ]:
model = VQAModel().to(device)
criterion = nn.CrossEntropyLoss(ignore_index=1)
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    # for parameter in model.parameters():
    #     parameter.requires_grad = False
    
    # model.eval()
    model.train()
    
    return model

model = load_checkpoint('/kaggle/input/model-vqa-nlp/model_ViT_PhoBert.pt').to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters.')

In [ ]:
vocab_swap = {value: key for key, value in vocab.items()}

In [ ]:
def evaluate(model, test_loader, criterion, vocab_swap, device, print_every=500, max_len=MAX_LEN):
    model.eval()  # Chuyển sang chế độ đánh giá
    test_losses = []
    total_loss = 0.0
    
    with torch.no_grad():  # Tắt tính toán gradient
        for batch_idx, batch in enumerate(tqdm(test_loader, desc="Evaluating")):
            anno_id, img_id, images, questions, answers = batch
        
            images = images.to(device)
            
            # Dự đoán
            predicted_tokens, ans_embedds = model(images, questions, answers, anno_id, mode='test', mask=False)
            predicted_tokens = predicted_tokens.float()
            ans_embedds = ans_embedds.long()
            
            # Tính loss
            loss = criterion(predicted_tokens.permute(0, 2, 1), ans_embedds)
            total_loss += loss.item()
            test_losses.append(loss.item())
            
            # In kết quả dự đoán và câu trả lời thực tế
            sentence_predicted = torch.argmax(predicted_tokens[i], axis=1)
            predicted_sentence = ""
            for idx in sentence_predicted:
                predicted_sentence += vocab_swap[idx.item()] + " "
                if idx == 2:  # Dừng khi gặp token kết thúc
                    break
            
            # In thông tin
            print(f"Batch [{batch_idx + 1}/{len(test_loader)}], Sample {i + 1}/{len(images)}")
            print(f"Question: {questions[i]}")
            print(f"Generated Answer: {predicted_sentence.strip()}")
            print(f"Actual Answer: {answers[i]}")
            print(f"Loss: {loss.item():.4f}")
            print("----------------------------")
    
        # In loss trung bình
        avg_loss = total_loss / len(test_loader)
        print(f"Average Test Loss: {avg_loss:.4f}")
    
    return test_losses, avg_loss

In [ ]:
test_losses, avg_loss = evaluate(model, test_loader, criterion, vocab_swap, device, print_every=500, max_len=MAX_LEN)